# Cvičenie 4: Monte Carlo metódy

Na predošlom cvičení sme videli, ako fungujú metódy Policy Iteration a Value Iteration, ktoré ale stále predpokladali úplnú znalosť dynamiky prostredia. Na dnešnom cvičení sa pozrieme na Monte Carlo metódy, konkrétne na on-policy first-visit Monte Carlo algoritmus odhadu $\pi^*$. Pracovať pritom budeme s rovnakým svetom gridworld.

## Metóda Monte Carlo

Na rozdiel od predošlých dvoch metód založených na dynamickom programovaní, Monte Carlo metódy sa učia z priamych interakcií s prostredím, práve preto ich priebeh závisí od konkrétnych epizódach interakcie. Vďaka tomu tieto metódy vieme aplikovať aj vtedy, ak nepoznáme úplny model, teda prechody medzi stavmi nám nie sú známe, vieme ich ale nasimulovať. Algoritmus zároveň pracuje explicitne s $\varepsilon$-soft politikou, ktorej príkladom je aj $\varepsilon$-greedy politika, teda akcia s najväčšou očakávanou hodnotou sa vyberie najčastejšie, ale s istou pravdepodobnosťou bude výber akcie náhodný.

Pseudokód algoritmu nájdete na obrázku nižšie, prediskutujte jeho kroky a význam premenných a parametrov, s ktorými algoritmus pracuje.

<img src="sources/lab04/monte_carlo.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Následne algoritmus aplikujeme pri riešení už známeho príkladu gridworld:

<img src="sources/lab04/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede svete. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra skončí. Pre ostatné kroky dostane odmenu -1.

Dolná a pravá časť sveta je úplne deterministická, pričom na pozíciach označených svetlomodrou farbou fúka silný vietor, ktorý môže agenta posunúť na východ aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na východ je $0.4$. Ak agent vyberie pohyb na východ, určite sa tam dostane. Ak sa pohybuje na západ, ostane na svojej pôvodnej pozícii (vietor a jeho pohyb sa rušia).

Discount factor $\gamma = 0.8$. Prvý odhad hodnoty pre každú dvojicu stav-akcia bude 0: $Q(s, a) = 0 \ \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$.

## Ukážka fungovania algoritmu

Majme epizódu:

$s_{33}, E, -1, s_{33}, S, -1, s_{33}, E, -1, s_{33}, E, -1, s_{33}, E, -1, s_{33}, N, -1, s_{23}, E, -1, s_{23}, E, -1, s_{23}, N, 10$,

ktorá predstavuje úspešnú interakciu, keďže agent sa dostal na cieľovú pozíciu.

Pri náhodnej politike ($\pi(a|s) = 0.25 \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$) a s hodnotami $\varepsilon = 0.1$  a $G=0$ sa táto epizóda spracuje nasledovne.

### Krok $s_{23}, N, 10$

Vyhodnotenie začneme od posledného kroku. Najprv sa vypočíta nová hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 0 + 10 = 10$

Ďalej zistíme, či sa kombinácia stavu a akcie $s_{23}, N$ nachádza v predošlých krokoch epizódy. Keďže takýto krok nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{23}, N) = \{ 10 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{23}, N) = \frac{\sum Returns(s_{23}, N)}{|Returns(s_{23}, N)|} = 10$

$Q(s_{23}) = \{ 10; 0; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou je teda $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{23}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

### Krok $s_{23}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 10 - 1 = 7$

Znova zisťujeme, či sa kombinácia stavu a akcie $s_{23}, E$ nachádza v predošlých krokoch epizódy. Keďže takýto krok bol vykonaný skôr v epizóde, ďalšie zmeny zatiaľ nerobíme.

### Krok $s_{23}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 7 - 1 = 4.6$

Znova zisťujeme, či sa kombinácia stavu a akcie $s_{23}, E$ nachádza v predošlých krokoch epizódy. Keďže takýto krok nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{23}, E) = \{ 4.6 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{23}, E) = \frac{\sum Returns(s_{23}, E)}{|Returns(s_{23}, E)|} = 4.6$

$Q(s_{23}) = \{ 10; 4.6; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou ostáva $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{23}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

### Krok $s_{33}, N, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 4.6 - 1 = 2.68$

Keďže krok $s_{33}, N$ nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{33}, N) = \{ 2.68 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{33}) = \{ 2.68; 0; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou bude $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{33}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

### Krok $s_{33}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 2.68 - 1 = 1.144$

Takýto krok bol vykonaný skôr niekoľkokrát v tejto epizóde, takže túto aktualizáciu urobíme až pri prvom výskyte tejto kombinácie v epizóde.

### Krok $s_{33}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 1.144 - 1 = -0.0848$

Takýto krok bol vykonaný skôr niekoľkokrát v tejto epizóde, takže túto aktualizáciu urobíme až pri prvom výskyte tejto kombinácie v epizóde.

### Krok $s_{33}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot (-0.0848) - 1 = -1.06784$

Takýto krok bol vykonaný skôr v tejto epizóde, takže túto aktualizáciu urobíme až pri prvom výskyte tejto kombinácie v epizóde.

### Krok $s_{33}, S, -1$

Nasleduje spracovanie druhého kroku epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot (-1.06784) - 1 = -1.854272$

Skúsenosť si zapamätáme, a aktualizujeme ostatné hodnoty:

$Returns(s_{33}, S) = \{ -1.854272 \}$

$Q(s_{33}) = \{ 2.68; 0; -1.85; 0 \}$

Akcia s najväčšou očakávanou hodnotou ostáva N.

### Krok $s_{33}, E, -1$

Ostáva nám prvý krok epizódy, ktorý je samozrejme prvým výskytom kombinácie $(s_{33}, E)$. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot (-1.854272) - 1 = -2.4834176$

Skúsenosť si zapamätáme, a aktualizujeme ostatné hodnoty:

$Returns(s_{33}, E) = \{ -2.48 \}$

$Q(s_{33}) = \{ 2.68; -2.48; -1.85; 0 \}$

Akcia s najväčšou očakávanou hodnotou ostáva N:

$\pi(N|s_{33}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

## Úloha:

Vypočítajte aktualizované hodnoty pomocou metódy Monte Carlo na nasledujúcich epizódach (svet je rovnaký):

$s_{33}, E, -1, s_{33}, E, -1, s_{33}, W, -1, s_{32}, W, -1, s_{31}, W, -1, s_{31}, W, -1, s_{31}, E, -1, s_{32}, S, -1, s_{32}, N, -10$

$s_{11}, S, -1, s_{12}, E, 10$

$s_{12}, N, -1, s_{12}, W, -1, s_{11}, W, -1, s_{11}, N, -1, s_{12}, E, 10$

Vaše riešenie si skontrolujte pomocou [ukážkovej implementácie metódy](sources/lab04/monte_carlo.py). Existuje možnosť, že metóda nájde optimálne riešenie už po prvej epizóde?

Môžete si vyskúšať aj [alternatívnu implementáciu metódy](sources/lab04/MCAgent.zip), ktorá prirodzenejšie modeluje interakciu medzi agentom a prostredím.